In [1]:
import guitarpro
import os
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from model import NotesRNN

In [65]:
folderpath = "tabs/necro/"
tab = guitarpro.parse(folderpath+'1 - Sweet trilogy part I.gp5')

In [3]:
def get_drum_notes(tab):
    
    for track in tab.tracks:
        if track.isPercussionTrack:
            break
    
    notes = []
    for measure in track.measures:
        for beat in measure.voices[0].beats:
            # pass empty measures
            if len(measure.voices[0].beats) == 1 and len(beat.notes) == 0:
                continue
            poly_note = ''
            for note in beat.notes:
                # replace different snare kicks
                if note.value == 40:
                    note.value = 38
                poly_note += str(note.value)
            notes.append(poly_note)
    
    return notes

In [4]:
notes = []
for filename in os.listdir(folderpath):
    tab = guitarpro.parse(folderpath + filename)
    notes += get_drum_notes(tab)

In [5]:
len(notes)

18233

In [8]:
dictionary = {note: i for i, note in enumerate(set(notes))}
inv_dictionary = {i: note for note, i in dictionary.items()}
notes_to_int = [dictionary[note] for note in notes]
notes_to_int[:10]

[152, 0, 26, 0, 37, 0, 49, 56, 26, 1]

In [9]:
notes[:10]

['4636', '', '44', '', '4438', '', '4436', '46', '44', '38']

In [10]:
def prepare_sequences(notes, dict_size, length, step):
    sequences = []
    targets = []
    for i in range(0, len(notes) - length, step):
        sequences.append(notes[i: i + length])
        targets.append(notes[i + length])
    targets = to_categorical(targets, dict_size)
    return sequences, targets

In [11]:
X, y = prepare_sequences(notes_to_int, len(dictionary), 32, 1)

In [43]:
RNN = NotesRNN(len(dictionary), 32, 128, 2, False, True)

In [44]:
RNN.model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 32)     5568        input_3[0][0]                    
__________________________________________________________________________________________________
lstm_5 (LSTM)                   (None, None, 128)    82432       embedding_2[0][0]                
__________________________________________________________________________________________________
lstm_6 (LSTM)                   (None, None, 128)    131584      lstm_5[0][0]                     
____________________________________________________________________________________________

In [41]:
X = np.array(X)
y = np.array(y)

In [45]:
RNN.model.fit(
    X, y,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    shuffle=True
)

Epoch 1/20
455/455 [==============================] - 31s 61ms/step - loss: 2.9340 - val_loss: 2.8702
Epoch 2/20
455/455 [==============================] - 28s 62ms/step - loss: 2.6261 - val_loss: 2.7958
Epoch 3/20
455/455 [==============================] - 28s 61ms/step - loss: 2.4916 - val_loss: 2.7311
Epoch 4/20
455/455 [==============================] - 26s 58ms/step - loss: 2.4246 - val_loss: 2.7401
Epoch 5/20
455/455 [==============================] - 26s 58ms/step - loss: 2.3709 - val_loss: 2.7218
Epoch 6/20
455/455 [==============================] - 27s 60ms/step - loss: 2.3041 - val_loss: 2.7162
Epoch 7/20
455/455 [==============================] - 27s 60ms/step - loss: 2.2516 - val_loss: 2.7334
Epoch 8/20
455/455 [==============================] - 25s 54ms/step - loss: 2.2089 - val_loss: 2.7453
Epoch 9/20
455/455 [==============================] - 26s 57ms/step - loss: 2.1948 - val_loss: 2.7665
Epoch 10/20
455/455 [==============================] - 26s 57ms/step - loss: 2.164

In [46]:
from tqdm.notebook import tqdm

In [47]:
def sample(preds, temperature):
    if temperature == 0:
        return np.argmax(preds)
    else:
        preds = np.log(preds) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        return np.random.choice(len(preds), p=preds)

In [48]:
temperature = 0.1
generated = notes_to_int[:32]

for i in tqdm(range(100)):
    preds = RNN.model.predict(generated[i:])[0]
    generated.append(sample(preds, temperature))

  0%|          | 0/100 [00:00<?, ?it/s]

In [49]:
generated_to_notes = [inv_dictionary[i] for i in generated]

In [341]:
song = guitarpro.Song()
track = guitarpro.models.Track(song, isPercussionTrack=True)
mes_header = guitarpro.models.MeasureHeader()

In [356]:
song = guitarpro.Song()
song.tracks[0].isPercussionTrack = True
song.tracks[0].measures[0].voices[0].beats.append(guitarpro.models.Beat(song.tracks[0].measures[0].voices[0]))

In [ ]:
strings = [0, 1, 2, 3, 4, 5, 6]
for i in range(len(generated_to_notes) // 16):
    measure = guitarpro.Measure(track, mes_header)
    measure.timeSignature.numerator = 16
    measure.timeSignature.denominator = 16
    note_sequence = generated_to_notes[i*16: (i+1)*16]
    for j in range(16):
        beat = guitarpro.Beat(measure.voices[0])
        if note_sequence[j] == '':
#             beat_note = guitarpro.Note(beat)
#             beat_note.type = guitarpro.models.NoteType(0)
#             beat.notes.append(beat_note)
            pass
        elif len(note_sequence[j]) > 2:
            for k in range(0, (len(note_sequence[j]) // 2) + 1, 2):
                beat_note = guitarpro.Note(beat)
                beat_note.value = int(note_sequence[j][k:k + 2])
                beat_note.string = strings[k // 2]
                beat_note.type = guitarpro.models.NoteType(1)
                beat.notes.append(beat_note)
        else:
            beat_note = guitarpro.Note(beat)
            beat_note.value = int(note_sequence[j])
            beat_note.type = guitarpro.models.NoteType(1)
            beat.notes.append(beat_note)
        measure.voices[0].beats.append(beat)
    song.tracks[0].measures.append(measure)
song.tracks[0].name = 'Drums'

In [357]:
strings = [0, 1, 2, 3, 4, 5, 6]
for i in range(len(generated_to_notes) // 16):
    measure = guitarpro.Measure(track, mes_header)
    measure.timeSignature.numerator = 16
    measure.timeSignature.denominator = 16
    note_sequence = generated_to_notes[i*16: (i+1)*16]
    for j in range(16):
        beat = guitarpro.Beat(measure.voices[0])
        if note_sequence[j] == '':
#             beat_note = guitarpro.Note(beat)
#             beat_note.type = guitarpro.models.NoteType(0)
#             beat.notes.append(beat_note)
            pass
        elif len(note_sequence[j]) > 2:
            for k in range(0, (len(note_sequence[j]) // 2) + 1, 2):
                beat_note = guitarpro.Note(beat)
                beat_note.value = int(note_sequence[j][k:k + 2])
                beat_note.string = strings[k // 2]
                beat_note.type = guitarpro.models.NoteType(1)
                beat.notes.append(beat_note)
        else:
            beat_note = guitarpro.Note(beat)
            beat_note.value = int(note_sequence[j])
            beat_note.type = guitarpro.models.NoteType(1)
            beat.notes.append(beat_note)
        measure.voices[0].beats.append(beat)
    song.tracks[0].measures.append(measure)
song.tracks[0].name = 'Drums'

In [358]:
len(song.tracks[0].measures)

9

In [340]:
song.tracks[0].measures[0].voices[0].beats

[]

In [311]:
track.measures[0].voices[0].beats = [guitarpro.Beat(track.measures[1].voices[0])]

In [312]:
track.measures[0].voices[0].beats

In [319]:
song.tracks[0] = track

In [359]:
guitarpro.write(song, 'generated.gp5')

GPException: writing measure 1, got AttributeError: 'int' object has no attribute 'value'

In [165]:
generated_to_notes[32:48]

['4936',
 '',
 '4438',
 '',
 '3654',
 '',
 '5147',
 '3536',
 '4438',
 '',
 '5147',
 '',
 '414336',
 '',
 '445136',
 '']